# Exercise 3

In [1]:
# optimization algorithm to minimize unconstrained function
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import matplotlib as plt
from scipy.optimize import newton

import plotly.express as px
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

Function we want to minimize

$f(x) = x + y$ 

$\nabla f(x) = (1, 1)^T$

constraints:
<br>
$h_1(x) = x_1x_2 - 1 \\
h_2(x) = x1^2 + 2x_2^2 + x_3^2 - 1$


In [2]:
# function to minimize
f = lambda x: x[0] + x[1]
# its gradient/jacobian
JacF = lambda x: [1, 1]
# norm of its gradient/jacobian
normJacF = lambda x: np.linalg.norm(JacF(x))
# constraint
h1 = lambda x: x[0] + x[1] + x[2] - 1
h2 = lambda x: x[0]**2 + 2*x[1]**2 + x[2]**2 - 1


Minimum of the problem

In [3]:
x_min = np.array((0,0,1))
print(f"{f(x_min)=}")
lambda1_lagrange = -1
lambda2_lagrange = 1/2

f(x_min)=0


# Penalty Method

$
\begin{equation}
\begin{aligned}
p(x) &= \frac{1}{2}||h(x)||_2^2 = \frac{1}{2}h_{1}(x)^Th_{1}(x) + \frac{1}{2}h_{2}(x)^Th_{2}(x) \\
&= \frac{1}{2}(x + y + z - 1)^T (x + y + z - 1) + \frac{1}{2}(x^{2} + 2y^{2} + z^{2} - 1)^T(x^{2} + 2y^{2} + z^{2} - 1)
\end{aligned}
\end{equation}
$

$
\begin{equation}
\begin{aligned}
P(x,\mu) &= f(x) + \mu p(x) \\ 
&= x + y + \frac{\mu}{2} \Big[(x + y + z - 1)^T (x + y + z - 1) + (x^{2} + 2y^{2} + z^{2} - 1)^T(x^{2} + 2y^{2} + z^{2} - 1)\Big]
\end{aligned}
\end{equation}
$

In [4]:
# feasibility penalization function
p = lambda x: 1/2 *( h1(x)**2 + h2(x)**2 )
# Merit function
P = lambda mu: lambda x: f(x) + mu*p(x)
JacP = lambda mu: lambda x: np.array([2*mu*x[0]*(x[0]**2 +2*x[1]**2 +x[2]**2 -1 ) + mu *(x[0]+x[1]+x[2]-1) +1 ,
                                     4*mu*x[1]*(x[0]**2 +2*x[1]**2 +x[2]**2 -1 ) + mu *(x[0]+x[1]+x[2]-1) +1,
                                     mu*(2*x[2]*(x[0]**2 +2*x[1]**2 +x[2]**2 -1) +x[0]+x[1]+x[2]-1)])

## Minimization algorithm

In [5]:
data = pd.DataFrame(columns=['iteration','x', 'f(x)',"||f'(x)||",'P(x)','p(x)','h1(x)','h2(x)','mu', 'mu*h1(x)', 'mu*h2(x)', 'log10||x_k-x_min||']).set_index('iteration')

# initial conditions
i=0 # iteration
x_start = [20,30,15]
mu_start = 5

x = x_start
mu = mu_start
mu_update_coef = 2

# start optimization loop
max_iter=100
while i <= max_iter:
    data.loc[i] = [x,f(x),normJacF(x),P(mu)(x),p(x),h1(x), h2(x), mu, mu*h1(x), mu*h2(x), np.log10(np.linalg.norm(x-x_min))]
    x = newton(JacP(mu), x, maxiter=10000,disp=False)
    mu += mu_update_coef
    i += 1 
data.tail(3)

,x,f(x),||f'(x)||,P(x),p(x),h1(x),h2(x),mu,mu*h1(x),mu*h2(x),log10||x_k-x_min||
iteration,,,,,,,,,,,
98,"[-0.004179109043989867, -0.002061227149985463,...",-0.00624034,1.414214,-0.003096,0.000016,-0.00500451,0.00249915,201,-1.00591,0.502328,-2.316874
99,"[-0.004151988925815282, -0.002026941234881167,...",-0.00617893,1.414214,-0.003065,0.000015,-0.00495507,0.00247468,203,-1.00588,0.50236,-2.320601
100,"[-0.004103829316961847, -0.0020143420745742107...",-0.00611817,1.414214,-0.003035,0.000015,-0.00490635,0.00245006,205,-1.0058,0.502262,-2.325191


## Plots

In [6]:
def save_plotly_fig_x_pure(fig, x_start,mu_start,mu_update):
    fig.write_image(f"media/Ex3_pure_xk_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}.png")
    
def save_plotly_fig_muh_pure(fig, x_start,mu_start,mu_update, lambda_used):
    fig.write_image(f"media/Ex3_pure_muh_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}_lambda={lambda_used}.png")
    
def save_plotly_fig_x_augmented(fig, x_start,mu_start,mu_update):
    fig.write_image(f"media/Ex3_aug_xk_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}.png")
    
def save_plotly_fig_muh_augmented(fig, x_start,mu_start,mu_update, lambda_used):
    fig.write_image(f"media/Ex3_aug_lambda_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}_lambda={lambda_used}.png")    

How is the convergence of x to $x_{min}$?

In [7]:
layout = dict(title_text='Convergence of x to the minimum in Pure penalization method', title_x=0.5, xaxis_title='iterations', yaxis_title='log10 ||x_k - x_min1||')

fig = px.scatter(data['log10||x_k-x_min||'][1:])
fig.update_layout(layout,showlegend=False) # add titles, remove unecessary legend

save_plotly_fig_x_pure(fig,x_start,mu_start,mu_update_coef)
fig.show()

Conclusions:

A lot of increases after iteration 23. Maybe matrix isn't semi - ND over there and newton is not able to perform well

In [8]:
# title information
layout = dict(title_text='behaviour of mu * h(x)', title_x=0.5, xaxis_title='iterations', yaxis_title='value')
# horizontal line with true value for lambda
lagrange_multiplier_line = dict(type= 'line', y0= lambda1_lagrange, y1= lambda1_lagrange, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))

fig = px.scatter(data['mu*h1(x)'][1:])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line with true value

save_plotly_fig_muh_pure(fig, x_start, mu_start, mu_update_coef, lambda1_lagrange)
fig.show()

In [9]:
# title information
layout = dict(title_text='behaviour of mu * h(x) in Augmented Lagrangian', title_x=0.5, xaxis_title='iterations', yaxis_title='mu*h(x)')
# horizontal line with true value for lambda
lagrange_multiplier_line = dict(type= 'line', y0= lambda2_lagrange, y1= lambda2_lagrange, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))

fig = px.scatter(data['mu*h2(x)'][1:])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line with true value

save_plotly_fig_muh_pure(fig, x_start, mu_start, mu_update_coef,lambda2_lagrange)
fig.show()

# Augmented Lagrangian Method

$
\begin{equation}
\begin{aligned}
L(x,\lambda,\mu) &= f(x) + \lambda^Th(x) + \mu \cdot p(x)\\
\end{aligned}
\end{equation}
$

In [10]:
# feasibility penalization function
p = lambda x: 1/2 * (h1(x)**2 + h2(x)**2)
# Merit function
L = lambda mu,lamb1,lamb2: lambda x: f(x) + lamb1*h1(x) + lamb2*h2(x) + mu*p(x)

JacL = lambda mu,lamb1,lamb2: lambda x: np.array([
    1 + lamb1 + 2*lamb2*x[0] + 2*mu*x[0]*(x[0]**2 +2*x[1]**2 +x[2]**2 -1 ) + mu *(x[0]+x[1]+x[2]-1),
    1 + lamb1 + 4*lamb2*x[1] + 4*mu*x[1]*(x[0]**2 +2*x[1]**2 +x[2]**2 -1 ) + mu *(x[0]+x[1]+x[2]-1),
    lamb1 + 2*lamb2*x[2] + mu*(2*x[2]*(x[0]**2 +2*x[1]**2 +x[2]**2 -1) +x[0]+x[1]+x[2]-1)])

## Minimization algorithm

In [11]:
data = pd.DataFrame(columns=['iteration','x', 'f(x)',"||f'(x)||",'L(x)','p(x)','h1(x)', 'h2(x)','mu','lambda1','lambda2', 'log10||x_k-x_min||']).set_index('iteration')
# initial conditions
i=0 # iteration
x_start = [20,30,15]
mu_start = 5
lambda1_start = 0
lambda2_start = 0

x = x_start
mu = mu_start
lamb1 = lambda1_start
lamb2 = lambda2_start
mu_update_coef = 2

last_saved_f = 0 # iteration to compare to in order to update mu when function has decreased enough


#begin optimization
max_iter=100
while i <= max_iter:
    data.loc[i] = [x, f(x), normJacF(x), L(mu,lamb1,lamb2)(x), p(x), h1(x), h2(x), mu, lamb1,lamb2, np.log10(np.linalg.norm(x-x_min))]
    x = newton(JacL(mu,lamb1,lamb2), x, maxiter=20000,disp=False)    
    
    # update mu
    if f(x) < 0.8 * data.loc[last_saved_f,'f(x)']:
        mu *= mu_update_coef
        last_saved_f = i
    lamb1 = lamb1 + mu * h1(x)
    lamb2 = lamb2 + mu * h2(x)
    i +=1
data.tail(3)

,x,f(x),||f'(x)||,L(x),p(x),h1(x),h2(x),mu,lambda1,lambda2,log10||x_k-x_min||
iteration,,,,,,,,,,,
98,"[-5.404709498274114e-09, 1.8908450500382514e-0...",1.35037e-08,1.414214,6.152399e-15,1.649368e-16,1.68692e-08,6.73087e-09,20,-1,0.5,-7.700022
99,"[2.3519084518085304e-08, -8.927919675418032e-0...",1.45912e-08,1.414214,1.039410e-14,1.072340e-16,1.46443e-08,1.06351e-10,20,-0.999999,0.5,-7.599347
100,"[-7.78828786089891e-08, 6.22230866872517e-08, ...",-1.56598e-08,1.414214,1.010117e-14,1.983452e-16,-1.88604e-08,-6.40121e-09,20,-1,0.5,-7.001139


## Plots

Convergence of x

In [12]:
layout = dict(title_text='Convergence of x to the minimum in Augmented Lagrangian method', title_x=0.5, xaxis_title='iterations', yaxis_title='log10 ||x_k - x_min||')
fig = px.scatter(data['log10||x_k-x_min||'])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend

save_plotly_fig_x_augmented(fig,x_start,mu_start,mu_update_coef)
fig.show()

Behaviour of mu*h(x)

In [13]:
# title information
layout = dict(title_text='behaviour of mu * h(x) in Augmented Lagrangian', title_x=0.5, xaxis_title='iterations', yaxis_title='mu*h(x)')
# horizontal line with true value for lambda
lagrange_multiplier_line = dict(type= 'line', y0= lambda1_lagrange, y1= lambda1_lagrange, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))

fig = px.scatter(data['lambda1'][1:])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line with true value

save_plotly_fig_muh_augmented(fig, x_start, mu_start, mu_update_coef,lambda1_lagrange)
fig.show()

In [14]:
# title information
layout = dict(title_text='behaviour of mu * h(x) in Augmented Lagrangian', title_x=0.5, xaxis_title='iterations', yaxis_title='mu*h(x)')
# horizontal line with true value for lambda
lagrange_multiplier_line = dict(type= 'line', y0= lambda2_lagrange, y1= lambda2_lagrange, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))

fig = px.scatter(data['lambda2'][1:])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line with true value

save_plotly_fig_muh_augmented(fig, x_start, mu_start, mu_update_coef,lambda2_lagrange)
fig.show()

Conclusions. The method converges very well to the first lagrangian from the start. But starts exploding after iteration 28

In [15]:
layout = dict(title_text='lambda2 estimate over the iterations', title_x=0.5,xaxis_title='iterations', yaxis_title='mu1 * h1(x)')

# horizontal line of correct lagrange multiplier
lagrange_multiplier_line = dict(type= 'line', y0= lambda2_lagrange, y1= lambda2_lagrange, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))
# title

fig = px.scatter(data,x=data.index,y='lambda2')
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line
fig.update_layout(layout) # add titles

